In [9]:
from mendeleev import element
import numpy as np
import re
import pandas as pd
import ast
from pymatgen.symmetry.groups import SpaceGroup
from mace.calculators import mace_mp
from ase import Atoms
import requests
from collections import Counter

# Define the function to calculate descriptors
def calculate_descriptors(formula):
    # Parse the formula and count the elements and their stoichiometric ratios
    pattern = re.compile(r"([A-Z][a-z]*)(\d*)")
    elements = pattern.findall(formula)

    # Convert elements list into a dictionary with counts as integers
    element_counts = {}
    for symbol, count in elements:
        count = int(count) if count else 1
        element_counts[symbol] = count

    # Retrieve each element's atomic properties
    atomic_numbers = []
    electronegativities = []
    atomic_radii = []
    atomic_masses = []
    valence_electrons = []

    for symbol, count in element_counts.items():
        elem = element(symbol)
        atomic_numbers.extend([elem.atomic_number] * count)
        electronegativities.extend([elem.en_pauling] * count)
        atomic_radii.extend([elem.atomic_radius] * count)
        atomic_masses.extend([elem.atomic_weight] * count)
        valence_electrons.extend([elem.nvalence()] * count)

    # Calculate descriptors

    # Elemental Properties
    mean_electronegativity = np.mean(electronegativities)
    atomic_number_diff = max(atomic_numbers) - min(atomic_numbers)
    avg_atomic_radius = np.mean(atomic_radii)

    # Compositional Features
    avg_atomic_mass = np.mean(atomic_masses)

    # Statistical Measures of Elemental Properties
    std_electronegativity = np.std(electronegativities)
    min_atomic_radius = np.min(atomic_radii)
    max_atomic_radius = np.max(atomic_radii)

    # Heuristic Quantities
    electronegativity_diff = max(electronegativities) - min(electronegativities)
    mean_valence_electron_count = np.mean(valence_electrons)

    # Organize descriptors into a vector
    feature_vector = [
        mean_electronegativity,        # 1. Mean Electronegativity
        atomic_number_diff,            # 2. Atomic Number Difference
        avg_atomic_radius,             # 3. Average Atomic Radius
        avg_atomic_mass,               # 4. Average Atomic Mass
        std_electronegativity,         # 5. Standard Deviation of Electronegativity
        min_atomic_radius,             # 6. Minimum Atomic Radius
        max_atomic_radius,             # 7. Maximum Atomic Radius
        electronegativity_diff,        # 8. Electronegativity Difference
        mean_valence_electron_count    # 9. Mean Valence Electron Count
    ]

    return feature_vector



def get_properties(row, ref):

    data = ast.literal_eval(row['structure'])
    
    lattice_data = data[0]['data']
    
    lat_a, lat_b, lat_c = lattice_data['a'], lattice_data['b'], lattice_data['c']
    
    volume = abs(np.dot(lat_a, np.cross(lat_b, lat_c))) * 1e30
    e_f = ast.literal_eval(row['formation_energy'])['value']
    
    atoms_data = lattice_data['atoms']

    # Create ASE Atoms object
    atoms = Atoms(
        symbols=[atom["element"] for atom in atoms_data],
        positions=[[atom["x"], atom["y"], atom["z"]] for atom in atoms_data],
        cell=[lat_a, lat_b, lat_c],
        pbc=True
    )

    # print(atoms.get_positions())
    atoms.set_cell(atoms.cell * 1e10)
    atoms.set_positions(atoms.positions * 1e10)
    # print(atoms.get_positions())

    atoms.calc = calc
    e_f_MLP = atoms.get_potential_energy()
    chemical_symbols = list(set(atoms.get_chemical_symbols())) 

    element_counts = Counter(atoms.get_chemical_symbols())
    for element in chemical_symbols:
        # print(e_f_MLP)
        e_f_MLP -= element_counts.get(element, 0) / ref[element][0] * ref[element][1]

    sg = SpaceGroup(row['space_group']).int_number
    
    return (volume, len(lattice_data['atoms']), sg, e_f, e_f_MLP)

In [10]:
# df = pd.read_csv('../Data/1_MatDX/MatDX_nomad_EF_Binary.csv')
# df = df[:5000]
df1 = pd.read_csv('../Data/1_MatDX/MatDX_nomad_EF_Binary.csv')
df2 = pd.read_csv('../Data/1_MatDX/MatDX_nomad_EF_Ternary.csv')

# Concatenate the DataFrames
df = pd.concat([df1, df2], ignore_index=True)

In [11]:
import json
from tqdm import tqdm

target = {}
descriptors = {}

calc = mace_mp(model="medium", dispersion=False, default_dtype='float64', device='cpu')

# Specify the file name
file_name = "reference_MACE.json"

# Read the JSON file into a dictionary
with open(file_name, 'r') as json_file:
    ref = json.load(json_file)

# Open the JSON file in append/update mode
output_file = 'descriptors_mace.json'
try:
    # Load existing data if the file already exists
    with open(output_file, 'r') as file:
        data = json.load(file)
except (FileNotFoundError, json.JSONDecodeError):
    # If the file doesn't exist or is empty, initialize an empty dictionary
    data = {}

# Loop through the DataFrame and process each row
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    try:
        formula = row['formula']
        (_, _, _, e_f, e_mlp) = get_properties(row, ref)
        target[formula] = e_f
        # descriptor_values = calculate_descriptors(formula)
        # descriptor_values += [e_mlp]
        descriptor_values = [index, e_mlp]
        descriptors[formula] = descriptor_values

        # Store the descriptor and e_f in the desired format
        data[formula] = [descriptor_values, e_f]
        
        # Save to JSON file after each entry (or periodically for large datasets)
        with open(output_file, 'w') as file:
            json.dump(data, file, indent=4)
            
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        pass

# Optionally, print the final number of descriptors processed
print("Total descriptors processed:", len(descriptors.keys()))


Using Materials Project MACE for MACECalculator with /Users/kisungkang/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


Processing Rows:   2%|▏         | 378/20000 [01:01<1:05:21,  5.00it/s]

Error processing row 378: list indices must be integers or slices, not str
Error processing row 379: list indices must be integers or slices, not str
Error processing row 380: list indices must be integers or slices, not str
Error processing row 381: list indices must be integers or slices, not str
Error processing row 382: list indices must be integers or slices, not str
Error processing row 383: list indices must be integers or slices, not str
Error processing row 384: list indices must be integers or slices, not str
Error processing row 385: list indices must be integers or slices, not str
Error processing row 386: list indices must be integers or slices, not str
Error processing row 387: list indices must be integers or slices, not str
Error processing row 388: list indices must be integers or slices, not str
Error processing row 389: list indices must be integers or slices, not str
Error processing row 390: list indices must be integers or slices, not str
Error processing row 391:

Processing Rows:   2%|▏         | 437/20000 [01:01<05:50, 55.85it/s]  

Error processing row 438: list indices must be integers or slices, not str
Error processing row 439: list indices must be integers or slices, not str
Error processing row 440: list indices must be integers or slices, not str
Error processing row 441: list indices must be integers or slices, not str
Error processing row 442: list indices must be integers or slices, not str
Error processing row 443: list indices must be integers or slices, not str
Error processing row 444: list indices must be integers or slices, not str
Error processing row 445: list indices must be integers or slices, not str
Error processing row 446: list indices must be integers or slices, not str
Error processing row 447: list indices must be integers or slices, not str
Error processing row 448: list indices must be integers or slices, not str
Error processing row 449: list indices must be integers or slices, not str
Error processing row 450: list indices must be integers or slices, not str
Error processing row 451:

...

Error processing row 17886: list indices must be integers or slices, not str
Error processing row 17887: list indices must be integers or slices, not str
Error processing row 17888: list indices must be integers or slices, not str
Error processing row 17889: list indices must be integers or slices, not str
Error processing row 17890: list indices must be integers or slices, not str
Error processing row 17891: list indices must be integers or slices, not str
Error processing row 17892: list indices must be integers or slices, not str
Error processing row 17893: list indices must be integers or slices, not str
Error processing row 17894: list indices must be integers or slices, not str
Error processing row 17895: list indices must be integers or slices, not str
Error processing row 17896: list indices must be integers or slices, not str
Error processing row 17897: list indices must be integers or slices, not str
Error processing row 17898: list indices must be integers or slices, not str

Processing Rows:  90%|█████████ | 18020/20000 [41:43<00:05, 336.67it/s]

Error processing row 18020: list indices must be integers or slices, not str
Error processing row 18021: list indices must be integers or slices, not str
Error processing row 18022: list indices must be integers or slices, not str
Error processing row 18023: list indices must be integers or slices, not str
Error processing row 18024: list indices must be integers or slices, not str
Error processing row 18025: list indices must be integers or slices, not str
Error processing row 18026: list indices must be integers or slices, not str
Error processing row 18027: list indices must be integers or slices, not str
Error processing row 18031: list indices must be integers or slices, not str
Error processing row 18032: list indices must be integers or slices, not str
Error processing row 18033: list indices must be integers or slices, not str
Error processing row 18034: list indices must be integers or slices, not str
Error processing row 18035: list indices must be integers or slices, not str

Processing Rows: 100%|██████████| 20000/20000 [46:48<00:00,  7.12it/s] 

Total descriptors processed: 9796
